# Merge Together the Stock Info

In [1]:
import os 
import pandas as pd
import numpy as np 

In [ ]:
all_data = []


for filename in os.listdir("data"):
    file_path = os.path.join("data", filename)
    df = pd.read_csv(file_path)
    df["Stock"] = filename[:-4]
    all_data.append(df)

merged_df = pd.concat(all_data, ignore_index=True)
merged_df["Date"] = pd.to_datetime(merged_df["Date"])

,Date,Open,High,Low,Close,Adj Close,Volume,Stock
0,2012-03-30,7.50,8.24,7.06,7.34,7.34,5048600.0,ENPH
1,2012-04-02,7.65,8.20,7.40,7.81,7.81,495500.0,ENPH
2,2012-04-03,7.75,7.80,7.50,7.64,7.64,316600.0,ENPH
3,2012-04-04,7.77,7.77,7.31,7.45,7.45,165100.0,ENPH
4,2012-04-05,7.48,7.55,7.37,7.43,7.43,104300.0,ENPH
...,...,...,...,...,...,...,...,...
237422,2022-01-19,35.60,36.61,34.76,35.07,35.07,18207400.0,VIAC
237423,2022-01-20,35.01,35.56,33.67,33.73,33.73,15664500.0,VIAC
237424,2022-01-21,32.70,32.82,31.13,31.25,31.25,21411800.0,VIAC
237425,2022-01-24,30.47,32.58,29.83,32.55,32.55,18814400.0,VIAC


# Make movie data mergable

In [12]:

df = pd.read_csv("movie_data/movies_with_details.csv")

df['Released'] = pd.to_datetime(df['Released'], errors='coerce', dayfirst=False)
df['Date'] = df['Released'].dt.strftime('%Y-%m-%d')
df = df.drop(columns=['Released'])



# Merge Stock and Movie Data

In [16]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
merged_df['Date'] = pd.to_datetime(merged_df['Date'], errors='coerce')

final_df = pd.merge(merged_df, df, on='Date', how='outer')  # or use 'outer', 'left', etc.

In [17]:
final_df.to_csv("merged_data/merged.csv")